In [1]:
import os
import spiceypy as spice
import json
import numpy as np
import pvl

import ale
from ale.base import Driver
from ale.base.label_isis import IsisLabel
from ale.base.data_naif import NaifSpice
from ale.base.type_distortion import RadialDistortion, NoDistortion
from ale.base.type_sensor import Framer, LineScanner
from ale.util import generate_kernels_from_cube
from ale.base.type_sensor import Framer
from ale.base.type_distortion import NoDistortion

from ale import util

In [2]:
class UvvisIsisLabelNaifSpiceDriver(Framer, IsisLabel, NaifSpice, NoDistortion, Driver):
    """
    Driver for reading Ultra-violet Invisible Spectrum ISIS3 Labels
    """
    
    @property
    def instrument_id(self):
        """
        Returns an instrument id for uniquely identifying the instrument,
        but often also used to be piped into Spice Kernels to acquire
        IKIDS. Therefor they are the same ID that Spice expects in bods2c
        calls. Expect instrument_id to be defined in the IsisLabel mixin.
        This should be a string of the form NEAR EARTH ASTEROID RENDEZVOUS

        Returns
        -------
        : str
          instrument id
        """
        lookup_table = {
        "UVVIS": "UVVIS"
        }
        return lookup_table[super().instrument_id]

    @property
    def sensor_name(self):
        """
        Returns the name of the instrument

        Returns
        -------
        : str
          instrument name
        """
        return "UVVIS"

    @property
    def sensor_model_version(self):
        """
        Returns ISIS sensor model version

        Returns
        -------
        : int
          ISIS sensor model version
        """
        return 1

    @property
    def spacecraft_clock_start_count(self):
        """
        The spacecraft clock start count, frequently used to determine the start time
        of the image.

        Returns
        -------
        : str
          spacecraft clock start count
        """
        if "SpacecraftClockStartCount" in self.label["IsisCube"]["Instrument"]:
            return str(
                self.label["IsisCube"]["Instrument"]["SpacecraftClockStartCount"])
        else:
            return None

    @property
    def spacecraft_clock_stop_count(self):
        """
        The spacecraft clock stop count, frequently used to determine the stop time
        of the image.

        Returns
        -------
        : str
          spacecraft clock stop count
        """
        if "SpacecraftClockStopCount" in self.label["IsisCube"]["Instrument"]:
            return str(
                self.label["IsisCube"]["Instrument"]["SpacecraftClockStopCount"])
        else:
            return None

    @property
    def ikid(self):
        """
        Overridden to grab the ikid from the Isis Cube since there is no way to
        obtain this value with a spice bods2c call. Isis sets this value during
        ingestion, based on the original fits file.

        Returns
        -------
        : int
          Naif ID used to for identifying the instrument in Spice kernels
        """
        return self.label["IsisCube"]["Kernels"]["NaifFrameCode"]

In [3]:
os.environ["ISISDATA"] = "/Users/ahibl/astro_efs"
os.environ["ISISTESTDATA"] = "Volumes/pkgs/isis3_old/isis_testData/"
os.environ["ISISROOT"] = "/Users/ahibl/working/astro/ISIS3/build/"

In [6]:
!spiceinit from=/Users/ahibl/astro_efs/test_imgs/lub0322a.054.cub

False False
Banana False False
Group = Kernels
  NaifFrameCode             = -40022
  LeapSecond                = $base/kernels/lsk/naif0012.tls
  TargetAttitudeShape       = Null
  TargetPosition            = $base/kernels/spk/de430.bsp
  InstrumentPointing        = ($Clementine1/kernels/ck/clem_2mn.bck,
                               $Clementine1/kernels/ck/clem_5sc.bck,
                               $clementine1/kernels/fk/clem_v12.tf)
  Instrument                = $Clementine1/kernels/ik/clem_uvvis_beta_ik_v04.-
                              ti
  SpacecraftClock           = $Clementine1/kernels/sclk/dspse002.tsc
  InstrumentPosition        = $Clementine1/kernels/spk/SPKMERGE_940219_940504-
                              _CLEMV001b.bsp
  InstrumentAddendum        = $Clementine1/kernels/iak/uvvisAddendum004.ti
  ShapeModel                = $base/dems/ldem_128ppd_Mar2011_clon180_radius_p-
                              ad.cub
  InstrumentPositionQuality = Reconstructed
  InstrumentPoin

In [5]:
# Generate kernels from an ISIS spiceinit'd label
cube = "/Users/ahibl/astro_efs/test_imgs/lua0048a.054.cub"
kernels = generate_kernels_from_cube(cube, expand=True, format_as='list')
kernels

TypeError: argument of type 'NoneType' is not iterable